# Linear Regression with PySpark

In [1]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('Linear').getOrCreate()

24/09/16 17:18:35 WARN Utils: Your hostname, Daniels-MacBook-Air-3.local resolves to a loopback address: 127.0.0.1; using 192.168.0.3 instead (on interface en0)
24/09/16 17:18:35 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/16 17:18:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
## read the dataset
df_pyspark=spark.read.option('header','true').csv('files/test1.csv',inferSchema=True)

In [3]:
df_pyspark.columns

['Name', 'age', 'Experience', 'Salary']

In [4]:
from pyspark.ml.feature import VectorAssembler

featureassembler = VectorAssembler(inputCols=['age', 'Experience'], 
                                   outputCol = 'Independent Features')

In [5]:
output=featureassembler.transform(df_pyspark)

In [6]:
output.show()

+---------+---+----------+------+--------------------+
|     Name|age|Experience|Salary|Independent Features|
+---------+---+----------+------+--------------------+
|    Krish| 31|        10| 30000|         [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|          [30.0,8.0]|
|    Sunny| 29|         4| 20000|          [29.0,4.0]|
|     Paul| 24|         3| 20000|          [24.0,3.0]|
|   Harsha| 21|         1| 15000|          [21.0,1.0]|
|  Shubham| 23|         2| 18000|          [23.0,2.0]|
+---------+---+----------+------+--------------------+



In [7]:
finalized_data=output.select('Independent Features', 'Salary')

In [8]:
finalized_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



In [15]:
from pyspark.ml.regression import LinearRegression

train_data, test_data=finalized_data.randomSplit([0.7, 0.3])
regressor = LinearRegression(featuresCol='Independent Features', labelCol='Salary')
regressor = regressor.fit(train_data)

24/09/16 17:20:00 WARN Instrumentation: [132c2838] regParam is zero, which might cause numerical instability and overfitting.


In [16]:
regressor.coefficients

DenseVector([47.619, 1285.7143])

In [17]:
regressor.intercept

13619.047619047662

In [18]:
pred_results=regressor.evaluate(test_data)

In [19]:
pred_results.predictions.show()

+--------------------+------+-----------------+
|Independent Features|Salary|       prediction|
+--------------------+------+-----------------+
|          [23.0,2.0]| 18000|17285.71428571428|
|         [31.0,10.0]| 30000|27952.38095238097|
+--------------------+------+-----------------+



In [21]:
pred_results.meanAbsoluteError, pred_results.meanSquaredError

(1380.9523809523762, 2351473.922902466)